In [0]:
import re
from datetime import datetime

ALL FUNCTIONS USED

In [0]:
def load(filepath):
    rdd = sc.textFile(filepath)
    return rdd

In [0]:
def load_data(x):
    return x.map(lambda x:re.sub(',(?=[^"]*"[^"]*(?:"[^"]*"[^"]*)*$)',"",x)).map(lambda x: x.split(","))

In [0]:
def data_split(x):
    return x.map(lambda x: x.split("|"))

In [0]:
def remove_header(x):
    header = x.first()
    return x.filter(lambda x: x != header)
    

In [0]:
def reduce(x):
    return x.map(lambda x:(x,1)).reduceByKey(lambda x1,x2 : x1+x2)

In [0]:
def sort(x):
    return x.sortBy(lambda x: x[1], ascending = False)

LOADING CLINICAL TRAIL DATA

In [0]:
clinicaltrial21 = load("/FileStore/tables/clinicaltrial_2021.csv/")
trial_data21=remove_header(clinicaltrial21)

meshrdd = load("/FileStore/tables/mesh.csv/")
mesh1 = remove_header(meshrdd)

pharma_rdd = load("/FileStore/tables/pharma.csv/")
pharma_1rdd = load_data(pharma_rdd)

FILTERING MESH CSV DATA

In [0]:

mesh1 = load_data(mesh1).map(lambda x: (x[0], x[1][:3])) 


FILTERING PHARMA CSV DATA

In [0]:

pharma_2rdd = pharma_1rdd.map(lambda x: x[1]).map(lambda x:x.strip('"'))
pharma_2rdd = reduce(pharma_2rdd)


IMPLEMENTATION OF QUESTION 1

In [0]:
trial_data21.distinct().count()

Out[11]: 387261

IMPLEMENTATION OF QUESTION 2

In [0]:
Type = data_split(trial_data21).map(lambda x: (x[5]))
Type21 = reduce(Type)
Type21=sort(Type21)
Type21.take(4)

Out[12]: [('Interventional', 301472),
 ('Observational', 77540),
 ('Observational [Patient Registry]', 8180),
 ('Expanded Access', 69)]

IMPLEMENTATION OF QUESTION 3

In [0]:
conditions_1 = data_split(trial_data21).flatMap(lambda x: (x[7].split(","))).filter(lambda x: len(x) !=0)
conditions_21=reduce(conditions_1)
conditions_21 =sort(conditions_21)
conditions_21.take(5)

Out[13]: [('Carcinoma', 13389),
 ('Diabetes Mellitus', 11080),
 ('Neoplasms', 9371),
 ('Breast Neoplasms', 8640),
 ('Syndrome', 8032)]

IMPLEMENTATION OF QUESTION 4

In [0]:

joinrdd = conditions_21.join(mesh1).map(lambda x: x[1]).map(lambda x: (x[1],x[0])).reduceByKey(lambda x1,x2: x1+x2)
joinrdd=sort(joinrdd)
joinrdd.take(10)

Out[14]: [('C04', 143994),
 ('C23', 136079),
 ('C01', 106674),
 ('C14', 94523),
 ('C10', 92310),
 ('C06', 85646),
 ('C08', 70720),
 ('C13', 42599),
 ('C18', 41276),
 ('C12', 40161)]

IMPLEMENTATION OF QUESTION 5

In [0]:
sponsor21 =data_split(trial_data21).map(lambda x: x[1]).map(lambda x:[x,1])
q5 = sponsor21.subtractByKey(pharma_2rdd).map(lambda x: (x[0],x[1])).reduceByKey(lambda x1,x2: x1+x2)
q5 = sort(q5)
q5.take(10)

Out[15]: [('National Cancer Institute (NCI)', 3218),
 ('M.D. Anderson Cancer Center', 2414),
 ('Assistance Publique - Hôpitaux de Paris', 2369),
 ('Mayo Clinic', 2300),
 ('Merck Sharp & Dohme Corp.', 2243),
 ('Assiut University', 2154),
 ('Novartis Pharmaceuticals', 2088),
 ('Massachusetts General Hospital', 1971),
 ('Cairo University', 1928),
 ('Hoffmann-La Roche', 1828)]

IMPLEMENTATION OF QUESTION 6

In [0]:
q6 = data_split(trial_data21).map(lambda x: (x[2], x[4][:3], x[4][4:8])).filter(lambda x: x[2]=="2021").filter(lambda x: x[0]=='Completed')
q6_result=q6.map(lambda x:x[1])
q6_result = reduce(q6_result).sortBy(lambda x:datetime.strptime(x[0], '%b'))
q6_result.take(12)

Out[16]: [('Jan', 1131),
 ('Feb', 934),
 ('Mar', 1227),
 ('Apr', 967),
 ('May', 984),
 ('Jun', 1094),
 ('Jul', 819),
 ('Aug', 700),
 ('Sep', 528),
 ('Oct', 187)]